In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
19251,2024-03-16,Romênia Divisão A,13:00,Arges Pitesti,Brasov,1.31,3.25,160.5,1.82,1.88,0.0,0.00,0.00,AcSAp3bP,0.763359,0.307692,0.549451,0.531915,0.071051,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,246.490,115.405735,0.468196,1.2,1.643168,1.369306,166.47,2.832,1.296850,0.457927,-13.0,87,93,1.48,1.79,0.000,361.978,0.601661,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-0.10,-0.020,-112.500000,0.000000,0.0,0.000000
19252,2024-03-16,República Tcheca Zbl Feminina,15:30,Brandys nad Labem F,Trutnov F,2.20,1.62,143.5,1.85,1.85,1.5,2.05,1.48,fFw102F3,0.454545,0.617284,0.540541,0.540541,0.071829,358.870,321.904072,0.896994,0.0,0.000000,NaN,783.90,5.340,4.640307,0.868971,-128.0,354.054,226.474691,0.639661,0.6,1.341641,2.236068,281.35,5.562,4.363161,0.784459,-32.0,67,85,11.70,3.31,227.874,261.372,0.214724,0.000000,0.228357,-1.26,-0.252,-4.761905,0.000000,0.0,0.000000,-5.00,-1.000,-0.620000,0.000000,0.0,0.000000
19253,2024-03-16,República Tcheca Zbl Feminina,13:00,Slavia Prague F,Hradec Kralove F,8.70,1.05,142.5,1.83,1.87,0.0,0.00,0.00,6VINnqo4,0.114943,0.952381,0.546448,0.534759,0.067323,285.272,388.620253,1.362280,1.2,1.643168,1.369306,185.60,3.876,4.728518,1.219948,-80.0,336.634,322.464970,0.957910,0.6,1.341641,2.236068,795.00,6.084,5.881686,0.966747,-92.0,58,53,3.20,15.00,142.392,226.310,1.109614,0.015289,NaN,-1.45,-0.290,-26.551724,0.000000,0.0,0.000000,-5.00,-1.000,-0.050000,0.000000,0.0,0.000000
19254,2024-03-16,República Tcheca Nbl,15:30,Ostrava,Olomoucko,1.43,2.68,163.5,1.84,1.86,0.0,0.00,0.00,xU9vhEMq,0.699301,0.373134,0.543478,0.537634,0.072435,132.942,46.204218,0.347552,1.8,1.643168,0.912871,107.16,1.538,0.353794,0.230035,32.0,134.126,37.484574,0.279473,2.4,1.341641,0.559017,87.36,1.502,0.463217,0.308400,31.0,76,78,1.41,1.12,117.278,189.444,0.430114,0.007644,NaN,0.48,0.096,4.479167,0.596146,0.8,0.203854,0.58,0.116,14.482759,0.000000,0.0,0.000000
19255,2024-03-16,República Tcheca Nbl,13:45,Kolin,Decin,3.08,1.34,166.5,1.86,1.84,0.0,0.00,0.00,MXRes3iR,0.324675,0.746269,0.537634,0.543478,0.070944,292.918,186.064413,0.635210,0.6,1.341641,2.236068,191.40,3.824,2.260958,0.591255,-67.0,175.144,129.188472,0.737613,1.8,1.643168,0.912871,182.91,2.170,1.408173,0.648927,-22.0,87,91,2.20,2.01,224.410,221.416,0.556727,0.007644,NaN,0.28,0.056,37.142857,0.000000,0.0,0.000000,-2.03,-0.406,-0.837438,0.512208,0.5,-0.012208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19516,2024-03-16,Europa Liga Letônia,10:00,Latvijas Universitate,TAL TECH,3.34,1.30,153.5,1.85,1.85,0.0,0.00,0.00,bXNILifq,0.299401,0.769231,0.540541,0.540541,0.068632,246.318,109.352131,0.443947,1.2,1.643168,1.369306,131.37,3.674,1.878731,0.511358,-17.0,200.994,99.439887,0.494741,1.8,1.643168,0.912871,128.70,2.444,1.204421,0.492807,18.0,87,99,1.51,1.30,280.720,522.534,0.621766,0.000000,NaN,-0.37,-0.074,-31.621622,0.000000,0.0,0.000000,-2.35,-0.470,-0.638298,0.356644,0.3,-0.056644
19517,2024-03-16,Europa Liga Letônia,10:00,VEF Riga,BC Kalev,1.60,2.24,155.5,1.88,1.79,-1.5,1.81,1.83,IXUUIkv2,0.625000,0.446429,0.531915,0.558659,0.071429

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
19279,15:00,Portugal Proliga,Illiabum,Santo André,1.58,Back Home
19294,03:00,Japão Liga Feminina W,Fujitsu F,Chanson V-Magic F,1.61,Back Home
19307,00:15,México Cibacopa,Venados de Mazatlan,Zonkeys de Tijuana,1.91,Back Home
19335,21:30,Eua Ncaa,Western Kentucky,UTEP,1.63,Back Home
19336,22:00,Eua Ncaa,Colorado,Oregon,1.76,Back Home
19377,09:00,Turquia Kbsl Feminina,Galatasaray F,Ormanspor F,1.55,Back Home
19395,20:00,Chile Lnb,Las Animas,Valdivia,1.99,Back Home
19426,16:45,Espanha Acb,MoraBanc Andorra,Tenerife,2.22,Back Home
19478,14:00,Hungria Nacional,Kormend,Szolnoki Olaj,1.67,Back Home
19492,15:00,Espanha Leb Plata,Almansa,Huelva,1.83,Back Home
